<a href="https://colab.research.google.com/github/Xiaowei0402/CS229_final_project/blob/main/CS229_new_feateng_consolidated.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Define dataset name and template type

**Please fill in before running the code**

In [ ]:
dataname = 'GSM4705209_293T_mNG_rep1.csv'
datatype = 'mNG'

# Running Feature engineering

## Some helper functions

In [ ]:
# get reverse complement sequence
def reverse_comp(seq):
  seq = seq[::-1].upper()
  rc_seq = []
  for i in seq:
    if i=='A':
      rc_seq.append("T")
    elif i=='T':
      rc_seq.append('A')
    elif i=='C':
      rc_seq.append('G')
    else:
      rc_seq.append('C')

  return ''.join(rc_seq)

## Import dataset

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import plotly.express as px
import copy

In [ ]:
# mount drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# load dataset
os.chdir('/content/drive/MyDrive/CS229_Final_Project/GSE155490_RAW')

# Load information of target strand
df_strand_info = pd.read_csv('target_strand_indicator.csv', header=[1])

# load info of template strand
template_B2 = reverse_comp(df_strand_info[df_strand_info.desc == 'perfect_ds'].seq.to_list()[0])
template_mNG = reverse_comp(df_strand_info[df_strand_info.desc == 'mNG_perfect_ds'].seq.to_list()[0])

template = {'B2': template_B2, 'mNG': template_mNG}

# read the specific dataset
df = pd.read_csv(dataname)

## Extract features

### Mapping the oligo sequence and result by barcode

In [ ]:
# this is tied to each result dataset
def map_index(df, df_info):
  map_dict = {}

  for i in range(len(df)):
    curr_bc = df.iloc[i].BC
    info_ind = df_info.index[df_info['barcode'] == curr_bc].to_numpy()[0]
    map_dict.update({i:info_ind})

  return map_dict

map_dict = map_index(df, df_strand_info)

### Create features

In [ ]:
def create_features():

  # create a new dataframe to store the extracted features, for each A
  # comp_seq_ind: the index of the sequence entry in the result dataset
  # A_pos: position of the A that we are focusing on, position starting at #1
  # distance_to_5: distance of this A to the 5' end of the dsRNA region
  # distance_to_3: distance of this A to the 3' end of the dsRNA region
  # global_GC_content: global GC content of this complementary strand
  # local_GC_content_21: the GC content calculated from a local 21-bp region (10bp upstream + 10bp downstream), based on domain knowledge
  # 'bulge-T/TTC/TTCTT/TTCTTCT': if the complementary strand belongs to the case of the bulge
  # bulgesite: 1 / where is the bulge as centered to this A
  # mismatch_site_num: how many mismatched sites (could be one mismatched site with multiple mismatches)
  # mismatch_num: how many bases are mismatched for each mismatch site
  # mismatch_site_1/2: 1 / where the mismatch is as centered to this A (1 is the one closer to 5' end and 2 closer to 3' end)
  # G_penality_1/2: calculated gibbs free energy penalty induced by each mutation site
  # editing_site_mismatch: if this A is mismatched
  # is_toC: if this editing site mismatch is a A:C mismatch
  # is_toG: if this editing site mismatch is a A:G mismatch
  # is_out_of_ds: if this A is outside of dsRNA region
  # y: percentage of editing efficiency of this A
  # delta_y: changes in percentage of editing efficiency of this A compared to the perfectly double stranded starting sequence
  features = ['comp_seq_ind', 'A_pos', 'distance_to_5', 'distance_to_3','global_GC_content','local_GC_content_21','bulge-T', 'bulge-TTC','bulge-TTCTT','bulge-TTCTTCT','bulgesite',
              'mismatch_site_num', 'mismatch_num','mismatch_site_1','G_penalty_1','mismatch_site_2','G_penalty_2', 'editing_site_mismatch','is_toC','is_toG','is_out_of_ds','y','delta_y']

  df_extracted_A = pd.DataFrame(columns = features)
  return df_extracted_A, features

### Helper functions

In [ ]:
# helper function: find where the mismatch is regarding the template strand
# returning indicies of the mismatch on the template strand

def find_mismatches(temp, comp):
  RC_comp = reverse_comp(comp)
  return [i for i in range(len(temp)) if temp[i] != RC_comp[i]]

In [ ]:
# helper function: calculate free energy penalty at each mismatch site
# free energy of each single base pair fetched from this paper: doi: 10.1261/rna.1734309
# Real RNA structure and energies are much more sublte, this is just a crude estimate
# Base pairing energies (kcal/mol): CG: -5.53; UA: -4.42; UG: -4.45; UU: -5.82; UC:-0.37; other base pair: 0
# input: string of template sequence; and the reversed string of mismatch complementary sequence

def calculate_G_penalty(temp,mis_rev):
  # larger G_penalty indicates weaker binding
  dict_pair_eng = {'C':-5.53,'G':-5.53, 'A':-4.42,'T':-4.42}
  dict_mis_eng = {'G':-4.45,'T':-5.82,'C':-0.37}

  G_penalty = 0

  for i in range(len(temp)):
    prev_energy = dict_pair_eng[temp[i]]
    if temp[i] == 'T':
      cur_energy = dict_mis_eng[mis_rev[i]]
    elif mis_rev[i] == 'T':
      cur_energy = dict_mis_eng[temp[i]]
    else:
      cur_energy = 0
    G_penalty += cur_energy - prev_energy

  return G_penalty

In [ ]:
# helper function: find out indicies of As on the template strand
def find_A_inds(seq):
   return [i for i in range(len(seq)) if seq.startswith('A', i)]

In [ ]:
# helper function: calculate GC content of a given sequence

def cal_GC_content(seq):
  return (seq.count('G') + seq.count('G'))/len(seq)

In [ ]:
# helper functin: calculate mismatch distance
# input mm_sites: only all the mutations in one site

def cal_mm_dist(mm_site, A_site):
  if mm_site[0] < A_site:
    return mm_site[-1]-A_site
  else:
    return mm_site[0]-A_site

### loop through dataset

In [ ]:
def split_by_A(df, df_strand_info, temp_seq = template[datatype]):
  df_extracted_A, features = create_features()
  map_dict = map_index(df, df_strand_info)

  A_num = temp_seq.count('A')
  A_inds = find_A_inds(temp_seq)

  # loop through each data point
  for i in range (len(df)):
    # define a flag for different types of the data point
    # 0: to be discarded; 1: bulge; 2: mismatch; 3: disrupt or extend; 4: perfect_ds
    typeflag = 0

    # ignore control entries
    if df_strand_info.iloc[map_dict[i]]['sample.ctrl'] == 'barcode_ctrl':
      continue

    curr_x = np.zeros([1,len(features)])[0]
    comp_seq = df_strand_info.iloc[map_dict[i]].seq
    comp_rev = comp_seq[::-1]

    curr_x[0] = i
    curr_x[4] = cal_GC_content(comp_seq)

    comp_desc = df_strand_info.iloc[map_dict[i]].desc
    comp_site_raw = df_strand_info.iloc[map_dict[i]].site

    # in the case of bulge
    if 'bulge' in comp_desc:
      typeflag = 1
      if comp_desc == 'bulge-T':
        curr_x[6] = 1
      elif comp_desc == 'bulge-TTC':
        curr_x[7] = 1
      elif comp_desc == 'bulge-TTCTT':
        curr_x[8] = 1
      elif comp_desc == 'bulge-TTCTTCT':
        curr_x[9] = 1
    else:
      # nobulge
      curr_x[10] = 0

    # in the case of mismatches
    if ('mismatch' in comp_desc) or ('Tto' in comp_desc):
      typeflag = 2
      if 'double' in comp_desc:
        # double mutation
        curr_x[11] = 2
        # all individual sites of mismatches
        # this will be useful later on
        mismatched_sites = find_mismatches(temp_seq,comp_seq)
        # number of mismatches
        mm_len = int(len(mismatched_sites) /2)
        curr_x[12] = mm_len
        # calculate energy penalty for the first site:
        curr_x[14] = calculate_G_penalty(temp_seq[mismatched_sites[0]:mismatched_sites[mm_len-1]+1],
                                        comp_rev[mismatched_sites[0]:mismatched_sites[mm_len-1]+1])
        # calculate energy penalty for the second site:
        curr_x[16] = calculate_G_penalty(temp_seq[mismatched_sites[mm_len]:mismatched_sites[-1]+1],
                                        comp_rev[mismatched_sites[mm_len]:mismatched_sites[-1]+1])
      else:
        # single mutation, including editing site specific change ('TtoC' or 'TtoG')
        curr_x[11] = 1
        curr_x[15] = 0
        # all individual sites of mismatches
        mismatched_sites = find_mismatches(temp_seq,comp_seq)
        # number of mismatches
        mm_len = int(len(mismatched_sites))
        curr_x[12] = mm_len
        # calculate energy penalty:
        curr_x[14] = calculate_G_penalty(temp_seq[mismatched_sites[0]:mismatched_sites[mm_len-1]+1],
                                        comp_rev[mismatched_sites[0]:mismatched_sites[mm_len-1]+1])
    else:
      # no mutation
      curr_x[13] = 0
      curr_x[15] = 0

    # in the case of disrupted or extended complementary
    complement_site_inds = [0, len(temp_seq)-1]
    if 'disrupt' in comp_desc or 'extend' in comp_desc:
      typeflag = 3
      if comp_desc == 'disrupt_from_beginning':
        complement_site_inds[0] = len(temp_seq)-1-int(comp_site_raw)
      elif comp_desc == 'disrupt_from_bulge':
        complement_site_inds[1] = len(temp_seq)-1-int(comp_site_raw)
      elif 'extend_into_mNG' in comp_desc:
        complement_site_inds[0] = - int(comp_desc[-2:])

    # if it's a perfect ds:
    if 'perfect' in comp_desc:
      typeflag =4

    # if it belongs to other category that we don't care about
    # skip this result entry
    if typeflag == 0:
      continue

    # Next, loop through each A
    for Ai in A_inds:
      # mostly what we need to do is to find out the distance of each modification to this A
      # also need to compute local GC content
      curr_Ai_x = copy.deepcopy(curr_x)
      curr_Ai_x[1] = Ai

      # Add y information
      curr_Ai_x[-2] = df.iloc[i]['A'+str(A_inds.index(Ai)+1)]

      # Add delta y
      # fetch data for perfectly complementary strand
      perfect_comp = df[df['desc'].str.contains("perfect")]
      perfect_edit = float(perfect_comp['A'+str(A_inds.index(Ai)+1)])
      curr_Ai_x[-1] = df.iloc[i]['A'+str(A_inds.index(Ai)+1)] - perfect_edit

      # compute local GC content
      # get the closest 20 bp sequence on the complementary strand
      slice_inds = [Ai-10, Ai+10]
      if slice_inds[0] < 0:
        slice_inds[1] = slice_inds[1] + abs(slice_inds[0])
        slice_inds[0] = 0
      elif slice_inds[1] > len(temp_seq)-1:
        slice_inds[0] = slice_inds[0] - (slice_inds[1] - len(temp_seq) +1)
        slice_inds[1] = len(temp_seq)-1

      # calculate GC content
      curr_Ai_x[5] = cal_GC_content(comp_rev[slice_inds[0]:slice_inds[1]+1])

      # Calculate distance to the ends of dsRNA region
      curr_Ai_x[2] = complement_site_inds[0] - Ai
      curr_Ai_x[3] = complement_site_inds[1] - Ai
      if curr_Ai_x[2] * curr_Ai_x[3] > 0:
        # if this A is out of complementarity
        curr_Ai_x[20] = 1

      # calculate site information
      if typeflag == 1:
        # if it's a bulge
        mi = int(comp_site_raw)
        if mi == Ai:
          curr_Ai_x[10] = 0
          curr_Ai_x[20] = 1
        else:
          curr_Ai_x[10] = 1/(mi - Ai)
      elif typeflag == 2:
        # if mismatch
        # first, if it's a editing site variant
        mi = mismatched_sites
        # if the mismatch happens on this A
        if Ai in mi:
          if ('Tto' in comp_desc) or curr_Ai_x[12] == 1:
            # mutation site length is 1
            curr_Ai_x[17] = 1
            if comp_desc == 'TtoC' or comp_rev[Ai] == 'C':
              curr_Ai_x[18] = 1
            elif comp_desc == 'TtoG' or comp_rev[Ai] == 'G':
              curr_Ai_x[19] = 1
          else:
            # longer mutation site length, treat this A as not in dsRNA
            curr_Ai_x[20] = 1

        # record the location of each mutation
        if curr_Ai_x[11] == 1:
          # single mutation:
          if Ai not in mi:
            curr_Ai_x[13] = 1/cal_mm_dist(mi, Ai)
        elif curr_Ai_x[11] == 2:
          # double mutation:
          if Ai not in mi[:int(len(mi)/2)]:
            curr_Ai_x[13] = 1/cal_mm_dist(mi[:int(len(mi)/2)], Ai)
          if Ai not in mi[int(len(mi)/2):]:
            curr_Ai_x[15] = 1/cal_mm_dist(mi[int(len(mi)/2):], Ai)

      # Add rows to the dataframe
      df_extracted_A.loc[len(df_extracted_A.index)] = curr_Ai_x

  return df_extracted_A

## Run on dataset

In [ ]:
result_df = split_by_A(df, df_strand_info, temp_seq = template[datatype])
os.chdir('/content/drive/MyDrive/CS229_Final_Project')
result_df.to_csv('converted_new_'+dataname)